# Introduction
Calculates the average number of pcs in various window sizes for each work in the corpus.

In [ ]:
from music21 import *
import statistics
import numpy as np
import pandas as pd
import statsmodels
import math

In [ ]:
properCorpus = corpus.corpora.LocalCorpus('DPhilCorpus')
properCorpus.existsInSettings
properCorpus.addPath('~/Desktop/Oxford/Corpus/Sounding Corpus (Movements)/XML/')
properCorpus.directoryPaths
properCorpus.save()
properCorpus.existsInSettings

In [33]:
pieceName = []
listOfWorks = properCorpus.search('Op.')
for el in listOfWorks:
    pieceName.append(el.sourcePath)

In [ ]:
list_of_total_info = []
for el in pieceName:
    thePiece = corpus.parse(el)
    fileTitle = str(el).replace("/Users/joshua ballance/Desktop/Oxford/Corpus/Sounding Corpus (Movements)/XML/", "").replace(".xml", "").replace(".mxl", "")
    opusNumber = fileTitle.replace("Op.", "").replace(" ", "").replace("i", "").replace("v", "")
    movementNumber = fileTitle.replace("Op.", "").replace(" ", "").replace("1", "").replace("2", "").replace("3", "").replace("4", "").replace("5", "").replace("6", "").replace("7", "").replace("8", "").replace("9", "").replace("0", "")
    chordifiedPiece = thePiece.chordify()
    chords = chordifiedPiece.flat.recurse().getElementsByClass('Chord')
    rests = chordifiedPiece.flat.recurse().getElementsByClass('Rest')
    listOfOffsetsandDurations = []
    for i in range(0, len(chords)):
        temp = []
        offset_position = chords[i].offset
        print(chords[i], chords[i].seconds)
        list_of_duration_values = []
        for j in chords:
            if j.offset < offset_position:
                list_of_duration_values.append(j.seconds)
        for k in rests:
            if k.offset < offset_position:
                list_of_duration_values.append(k.seconds)
        listOfOffsetsandDurations.append([sum(list_of_duration_values), chords[i].pitches, chords[i].seconds])
    for i in range(0, len(rests)):
        temp = []
        offset_position = rests[i].offset
        list_of_duration_values = []
        for j in chords:
            if j.offset < offset_position:
                list_of_duration_values.append(j.seconds)
        for k in rests:
            if k.offset < offset_position:
                list_of_duration_values.append(k.seconds)
        listOfOffsetsandDurations.append([sum(list_of_duration_values), 'rest', rests[i].seconds])        
    listOfOffsetsandDurations = sorted(listOfOffsetsandDurations, key=itemgetter(0))
    all_chord_durations = []
    all_rest_durations = []
    for l in chords:
        all_chord_durations.append(l.seconds)
    for m in rests:
        all_rest_durations.append(m.seconds)
    temp = []
    temp.append(fileTitle)
    temp.append(listOfOffsetsandDurations)
    temp.append(sum(all_chord_durations)+sum(all_rest_durations))
    list_of_total_info.append(temp)

In [ ]:
all_pieces_list_of_pc_distribution_durations = []
all_pieces_list_of_pc_distribution_weighted_frequencies = []
all_pieces_list_of_pc_distribution_unweighted_frequencies = []
for h in range(0,len(list_of_total_info)):
    piece_title = list_of_total_info[h][0]
    print(piece_title)
    list_of_items = list_of_total_info[h][1]
    final_item = list_of_items[len(list_of_items)-1]
    max_offset = final_item[0]+final_item[2]
    highest_value = math.ceil(max_offset)
    this_piece_list_of_pc_distribution_durations = []
    this_piece_list_of_pc_distribution_weighted_frequencies = []
    this_piece_list_of_pc_distribution_unweighted_frequencies = []
    for i in range(1, highest_value+1): 
        window_size = i
        this_movement_magnitudes = []
        number_of_windows = math.ceil(max_offset/window_size)
        final_window_start_point = max_offset-window_size
        window_start_points = [0]
        for el in range(1, number_of_windows):
            if number_of_windows > 1:
                this_window_start_point = final_window_start_point/(number_of_windows-1)*el
                window_start_points.append(this_window_start_point)
        window_pitch_classes = []
        list_of_pc_distribution_durations = []
        list_of_pc_distribution_weighted_frequencies = []
        list_of_pc_distribution_unweighted_frequencies = []
        for j in window_start_points:
            this_window_contents = [j]
            window_start = j
            window_end = window_start+window_size
            for k in range(0, len(list_of_items)-1):
                this_item = list_of_items[k]
                if k == 0:
                    pass
                else:
                    prev_item = list_of_items[k-1]
                if k == len(list_of_items)-1:
                    next_item = [this_item[0]+this_item[2]]
                else:
                    next_item = list_of_items[k+1]
                #if the entire duration of the chord/rest is within the window
                if this_item[0] >= window_start:
                    if next_item[0] <= window_end:
                        if this_item[1] == 'rest':
                            pass
                        else:
                            for l in this_item[1]:
                                this_window_contents.append([l, this_item[2]])
                #if the chord/rest begins before window_start but continues past window_start and stops before or with window_end
                if this_item[0] < window_start:
                    if this_item[0]+this_item[2] > window_start: 
                        if next_item[0] <= window_end:
                            if this_item[1] == 'rest':
                                pass
                            else:
                                for l in this_item[1]:
                                    this_window_contents.append([l, this_item[2]-window_start+this_item[0]])
                #if the chord/rest begins before window_start but continues past window_start and stops after window_end
                if this_item[0] < window_start:
                    if next_item[0] > window_end:
                        if this_item[1] == 'rest':
                            pass
                        else:
                            for l in this_item[1]:
                                this_window_contents.append([l, window_size])
                #if the chord/rest begins after or with window_start but continues past window_start and stops past window_end
                if this_item[0] >= window_start:
                    if this_item[0] < window_end:
                        if next_item[0] > window_end:
                            if this_item[1] == 'rest':
                                    pass
                            else:
                                for l in this_item[1]:
                                    this_window_contents.append([l, this_item[2]-this_item[0]-this_item[2]+window_end])
            ##produce two pc distributions for this window, one measuring total duration of each pc, one measuring total frequency of each pc
            list_of_pitch_classes = []
            for m in this_window_contents[1:]:
                exact_pitch = m[0]
                pitch_class = exact_pitch.pitchClass
                duration = m[1]
                pc_data = [pitch_class, duration]
                list_of_pitch_classes.append(pc_data)
            pc_distribution_duration = []
            pc_distribution_weighted_frequency = []
            pc_distribution_unweighted_frequency = []
            for n in range(0, 12):
                this_pc_duration = [0]
                this_pc_frequency = [0]
                for o in list_of_pitch_classes:
                    if o[0] == n:
                        this_pc_duration.append(o[1])
                        this_pc_frequency.append(1)
                pc_distribution_duration.append(sum(this_pc_duration))
                pc_distribution_weighted_frequency.append(sum(this_pc_frequency))
                if this_pc_frequency == [0]:
                    pc_distribution_unweighted_frequency.append(0)
                else:
                    pc_distribution_unweighted_frequency.append(1)
            list_of_pc_distribution_durations.append(pc_distribution_duration)
            list_of_pc_distribution_weighted_frequencies.append(pc_distribution_weighted_frequency)
            list_of_pc_distribution_unweighted_frequencies.append(pc_distribution_unweighted_frequency)
        this_piece_list_of_pc_distribution_durations.append(list_of_pc_distribution_durations)
        this_piece_list_of_pc_distribution_weighted_frequencies.append(list_of_pc_distribution_weighted_frequencies)
        this_piece_list_of_pc_distribution_unweighted_frequencies.append(list_of_pc_distribution_unweighted_frequencies)
    all_pieces_list_of_pc_distribution_durations.append(this_piece_list_of_pc_distribution_durations)
    all_pieces_list_of_pc_distribution_weighted_frequencies.append(this_piece_list_of_pc_distribution_weighted_frequencies)
    all_pieces_list_of_pc_distribution_unweighted_frequencies.append(this_piece_list_of_pc_distribution_unweighted_frequencies)

In [ ]:
all_pieces_average_data = []
for i in all_pieces_list_of_pc_distribution_unweighted_frequencies:
    this_piece_data = i
    this_piece_average_data = []
    for j in range(0, len(this_piece_data)):
        window_size = j+1
        this_window_data = this_piece_data[j]
        this_window_vals = []
        for k in this_window_data:
            this_window_vals.append(sum(k))
        this_window_average_data = np.mean(this_window_vals)
        this_piece_average_data.append(this_window_average_data)
    all_pieces_average_data.append(this_piece_average_data)
all_pieces_average_data
titles = []
for h in range(0,len(list_of_total_info)):
    piece_title = list_of_total_info[h][0]
    titles.append(piece_title)
df = pd.DataFrame(all_pieces_average_data, index=titles)    
df = df.transpose()
df